In [4]:
import numpy as np
import gzip
import os
import tensorflow as tf
import re

# specify the path to the zipped file.
os.chdir('/mnt/c/Users/amsj1/OneDrive - University of Tennessee/2nd_year/BZAN554_deep_learning/bzan554-group-assignment1')

In [5]:
def parse(path):
    """
    Function to read in the VERY LARGE dataset and yield it as a generator for
    memory efficiency. Takes one argument which is the path to the file being
    read in. This path is set abose using 'os.chdir'.
    """
    with gzip.open(path, 'rb') as g:
        for l in g:
            yield eval(l)

#######################################################################
# Basically load in some data to make lookup tables for the sequentially
# learning part
#######################################################################
X_raw = []
Y_raw = []
counter = 0
for d in parse('meta_Clothing_Shoes_and_Jewelry.json.gz'):
    counter += 1
    X_raw.append(d['title'])
    Y_raw.append(d['category'])
    if counter == 100:
        break

X_words = []
for titles in X_raw:
    # split the titles into words and lowercase
    words = titles.lower().split()
    for word in words:
        # extract out words with minimal to no symbols
        stripped_word = re.sub('[0-9,.,#,\-,&,;,\',(,),/,@,+,!]', '', word)
        X_words.append(stripped_word)

X_words = np.unique(np.array(X_words))

# look up for X
lookup_X = []
lookup_X.extend([word for word in X_words])

# flatten the Y list of lists
flat_Y = [category for subcat in Y_raw for category in subcat]
# uniquefy it
unique_Y = np.unique(np.array(flat_Y))

# look up for Y
lookup_Y = []
lookup_Y.extend([category for category in unique_Y])

In [6]:
#######################################################################
# Set up the model architecture
#######################################################################
inputs = tf.keras.layers.Input(shape = (len(X_words),))
hidden1 = tf.keras.layers.Dense(
    units = 2,
    activation = 'sigmoid',
    name = 'hidden1'
)(inputs)
hidden2 = tf.keras.layers.Dense(
    units = 2,
    activation = 'sigmoid',
    name = 'hidden2'
)(hidden1)
output = tf.keras.layers.Dense(
    units = len(unique_Y),
    activation = 'softmax',
    name = 'output'
)(hidden2)

#######################################################################
# create & compile the model
#######################################################################
model = tf.keras.Model(inputs = inputs, outputs = output)
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = tf.keras.optimizers.SGD(learning_rate = 0.001)
)

2021-09-21 00:13:42.260678: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: UNKNOWN ERROR (100)
2021-09-21 00:13:42.260723: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-U49P9H1): /proc/driver/nvidia/version does not exist
2021-09-21 00:13:42.260947: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
#######################################################################
# Big boy model learning time
#######################################################################
counter = 0
# for i in range(1000): # number of epochs
for d in parse('meta_Clothing_Shoes_and_Jewelry.json.gz'): # instances
    counter += 1
    X_raw = np.array(d['title'])
    Y_raw = np.array(d['category'])

    ###############################################
    # logic for the X hot-one-encoded stuff
    ###############################################
    # split the titles into words and lowercase
    # words = word.lower().split()
    X_words = [words for words in str(X_raw).lower().split()]

    # Get final binary stuff for X
    X_indices = np.where(np.isin(lookup_X, X_words))
    X_final = np.zeros(len(lookup_X))
    X_final[X_indices] = 1
    X_final = X_final.reshape(1, len(X_final))

    ###############################################
    # logic for the Y hot-one-encoded stuff
    ###############################################
    Y_indices = np.where(np.isin(lookup_Y, unique_Y))
    Y_final = np.zeros(len(lookup_Y))
    Y_final[Y_indices] = 1
    Y_final = Y_final.reshape(1, len(Y_final))

    ###############################################
    # Finallllyyyyy fit the freakin model
    ###############################################
    model.fit(x=X_final,y=Y_final,epochs=1,batch_size=1)

    if counter == 100:
        break

    # break

2021-09-21 00:13:42.538317: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


1/1 [==============================] - 0s 3ms/step - loss: 2091.7241


In [8]:
yhat = model.predict(x=X_final)
yhat

array([[0.00277744, 0.00281993, 0.00265229, 0.00299162, 0.00273926,
        0.00263211, 0.00283557, 0.00280701, 0.00266955, 0.00275117,
        0.00279161, 0.00293741, 0.00284017, 0.00278731, 0.00270321,
        0.00293271, 0.00288327, 0.00260372, 0.00274327, 0.00277   ,
        0.00294831, 0.00281733, 0.0030189 , 0.00290596, 0.00269125,
        0.0029899 , 0.00283357, 0.00276417, 0.00293417, 0.00298697,
        0.00278153, 0.00274223, 0.00264432, 0.00264304, 0.00290258,
        0.00300501, 0.0028368 , 0.00281739, 0.00278779, 0.0028681 ,
        0.00274971, 0.00260692, 0.00265254, 0.00281952, 0.0029725 ,
        0.00301883, 0.00281708, 0.00293136, 0.00268518, 0.00282762,
        0.00260922, 0.00297631, 0.00271172, 0.00282328, 0.00267438,
        0.00288217, 0.0028537 , 0.0026958 , 0.00285873, 0.00278753,
        0.00277437, 0.002989  , 0.00273626, 0.002877  , 0.002806  ,
        0.00273662, 0.00272129, 0.00286757, 0.00284113, 0.00292489,
        0.00265789, 0.00285031, 0.00282462, 0.00